In [8]:
!conda install pytorch torchvision -c pytorch

Channels:
 - pytorch
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [1]:
from S3DogDataset import *
from torch.utils.data import DataLoader, RandomSampler
from torchvision import models
from tqdm import tqdm
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
train_dataset = S3DogDataset(file_ref_csv='train_labels.csv')
test_dataset = S3DogDataset(file_ref_csv='test_labels.csv')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

train_sampler = RandomSampler(train_dataset, replacement=True, num_samples=1000)
test_sampler = RandomSampler(test_dataset, replacement=True, num_samples=100)

train_sample_loader = DataLoader(train_dataset, batch_size=4, sampler=train_sampler)
test_sample_loader = DataLoader(test_dataset, batch_size=4, sampler=test_sampler)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
num_classes = 120

mobilenet_v3_large = models.mobilenet_v3_large(weights="IMAGENET1K_V2")

for param in mobilenet_v3_large.parameters():
    param.requires_grad = False

num_features = mobilenet_v3_large.classifier[3].in_features
mobilenet_v3_large.classifier[3] = torch.nn.Linear(num_features, num_classes)
            
model = mobilenet_v3_large

In [4]:
for param in model.parameters():
    param.requires_grad = True
                
if torch.cuda.is_available():
    device = torch.device('cuda') 
else:
    device = torch.device('cpu')
print(device)

model = model.to(device)

cpu


In [5]:
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
train_loader = train_sample_loader
test_loader = test_sample_loader

for epoch in range(num_epochs):
    epoch_start = time.time()
    model.train()
    for i, (images, labels) in enumerate(tqdm(train_loader, desc='Training')):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
                      
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(tqdm(test_loader, desc='Testing')):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print('Accuracy: {} % | Epoch Time: {}'.format(100 * correct / total, time.time()-epoch_start))
     
        torch.save(model.state_dict(), 'model.pth')
        print('Model saved.')

Training: 100%|██████████| 250/250 [04:00<00:00,  1.04it/s]


Epoch [1/10], Step [250/250], Loss: 4.7221


Testing: 100%|██████████| 25/25 [00:15<00:00,  1.62it/s]


Accuracy: 2.0 % | Training Time: 256.07140588760376
Model saved.


Training:  15%|█▌        | 38/250 [00:34<03:12,  1.10it/s]


KeyboardInterrupt: 